In [ ]:
import json
import pandas as pd

## Ribonanza

## RNAStralign

In [104]:
data = json.load(open('../RNAStralign/data.json'))
families = pd.DataFrame.from_dict(data, orient='index')['family'].apply(lambda x: x.split('__')[0].replace('_database', '')).value_counts()
families_rnastralign = families

## archiveII

In [105]:
data = json.load(open('../archiveII/data/archiveII/data.json'))
families = pd.DataFrame.from_dict(data, orient='index').reset_index()['index'].apply(lambda x: x.split('_')[0]).value_counts()
families

index
5s            1149
srp            725
tRNA           493
RNaseP         429
tmRNA          404
16s            110
grp1            93
telomerase      37
23s             35
grp2            11
Name: count, dtype: int64

In [108]:
for k in (families_rnastralign + families).keys():
    print(k, end=' ')

16S_rRNA 16s 23s 5S_rRNA 5s RNaseP SRP group_I_intron grp1 grp2 srp tRNA telomerase tmRNA 

## bpRNA

In [50]:
data = json.load(open('../bpRNA/data/bpRNA/data.json'))
df = pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'reference'})
df['family'] = df['reference'].apply(lambda x: x.split('_')[1])
df.set_index('reference', inplace=True)

In [ ]:
df['family'].value_counts()

### bpRNA/CRW

In [12]:
# Make a fasta for the CRW sequences
def df_to_fasta(data, path):
    data = data.to_dict(orient='index')
    with open(path, 'w') as f:
        f.write('\n'.join('>{}\n{}'.format(name, data[name]['sequence']) for name in data))
df_to_fasta(df[df['family'] == 'CRW'], 'bprna_crw.fasta')
df_to_fasta(df[df['family'] == 'RFAM'], 'bprna_rfam.fasta')


In [ ]:
# make a blast database for crw
!makeblastdb -in ../CRW/crw.fasta -dbtype nucl -out db/crw

In [ ]:
# run bprna_crw against crw
!blastn -query bprna_crw.fasta -db db/crw -outfmt 6 -out blast_crw.csv

In [27]:
df = pd.read_csv('blast_rfam.csv', sep='\t', header=None)
df.columns = ['query', 'subject', 'identity', 'length', 'mismatches', 'gaps', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
df['family'] = df['subject'].apply(lambda x: x.split('___')[0])

### bpRNA/RFAM

In [37]:
df.groupby('query')['family'].unique().apply(len).value_counts()

family
1    1469
2      25
3       1
Name: count, dtype: int64

In [34]:
len(df['query'].unique())

1495

In [56]:
rfam = pd.read_csv('/Users/yvesmartin/src/supermodels-data/rfam/data/rfam.csv')

In [57]:
rfam

,family,reference,ref_desc,sequence,family_name,full_family_name,clan_name
0,RF03636,AEWC01008534.1/27091-27004,Solanum tuberosum cultivar DM 1-3 516 R44 scf0...,TACGCAGGAGAGATGATGCTGGACGATAGAGATATGAGAATATTAA...,MIR4376,MIR4376 microRNA precursor family,NaN
1,RF03636,CM001069.2/4541889-4541964,Solanum lycopersicum cultivar Heinz 1706 chrom...,TACGCAGGAGAGATGATGCTGGACGATAGAGAAAATGGGATTTAAT...,MIR4376,MIR4376 microRNA precursor family,NaN
2,RF03636,CM002817.1/83173031-83173111,"Capsicum annuum cultivar Zunla-1 chromosome 6,...",TACGCAGGAGAGATGATGCTGGACGATAGAGATAGGAATAATGGGC...,MIR4376,MIR4376 microRNA precursor family,NaN
3,RF03636,AYMY01S002128.1/82671-82774,Nicotiana tabacum cultivar TN90 Ntab-TN90_scaf...,TACGCAGGAGAGATGATGCTGGATAATGGAGCATCCCTACTTAGCT...,MIR4376,MIR4376 microRNA precursor family,NaN
4,RF03636,MJEQ01004217.1/14857-14960,"Nicotiana attenuata strain UT scaffold04217, w...",TACGCAGGAGAGATGATGCTGGATAATGGACCATCCCTACTTAGCT...,MIR4376,MIR4376 microRNA precursor family,NaN
...,...,...,...,...,...,...,...
3081743,RF01532,AAHA01000001.1/3813320-3813388,Trypanosoma brucei brucei strain 927/4 GUTat10...,CGCCAGCAGCCGCCCCGGGCAGGAUGGCGCCCGUGUCCUUGGCGUG...,TB10Cs3H2,Trypanosomatid snoRNA TB10Cs3H2,NaN
3081744,RF01532,FR796465.1/103544-103472_1,Leishmania infantum JPCM5 genome chromosome 33,UGCCGCCUGCACCUUCCGGGCAGGAUGCCGCCCUUGCGCCCUUGGC...,TB10Cs3H2,Trypanosomatid snoRNA TB10Cs3H2,NaN
3081745,RF01532,FR799008.1/88022-87950_1,Leishmania braziliensis MHOM/BR/75/M2904 compl...,UGCUGCCUGCACCUUCCGGGCAGGAUGACGCCCUUGCGGCCUUGGC...,TB10Cs3H2,Trypanosomatid snoRNA TB10Cs3H2,NaN
3081746,RF01532,FN554973.1/3774444-3774512,Trypanosoma brucei gambiense DAL972 chromosome...,CGCUAGCAGCCGCCCCGGGCAGGAUGGCGCCCGUGUCCUUGGCGUG...,TB10Cs3H2,Trypanosomatid snoRNA TB10Cs3H2,NaN


In [52]:
bp_rfam = pd.DataFrame(open('/Users/yvesmartin/src/supermodels-data/family-classification/bprna_rfam.fasta').read().split('>')[1:], columns=['sequence'])   
bp_rfam['reference'] = bp_rfam['sequence'].apply(lambda x: x.split('\n', 1)[0].replace('\n', ''))
bp_rfam['sequence'] = bp_rfam['sequence'].apply(lambda x: x.split('\n', 1)[1].replace('\n', ''))
bp_rfam

,sequence,reference
0,GGGGAUGACUGGUUUCGACUUGGAAUGGUUGUUCCAGGAGAAGCGG...,bpRNA_RFAM_3948
1,GGUCAUGAGUGCCAGCGACAAGCCCCGGCUCGCUGGCCGGCAACCC...,bpRNA_RFAM_18423
2,AUACAUACCUGGCCGGGGGAAGUCGGUGAUCAAGACGGCCGAGUCC...,bpRNA_RFAM_838
3,AAAUGGUGGCCCGGCGUUGGUUCCUGUCCUAUGACAGGCGAAGAGG...,bpRNA_RFAM_9155
4,GGACUUGGGCUUGAGGGGAGGGGUGCUCUCUGUGCCUGAGGGACAU...,bpRNA_RFAM_20831
...,...,...
39596,AGGAUAAAAACUGGGGGGUAUUAGAAGUCAAGAAAAUAAUAGAUGC...,bpRNA_RFAM_39718
39597,AGGAGACCAGCCCCGCCUUCCCAGUGUUCAGACUACCUGUUCAGGA...,bpRNA_RFAM_7639
39598,UGUGCAUCGUGGUCAAAUGCUCAGACUCCUGUGGUGGCUGCUCAUG...,bpRNA_RFAM_19577
39599,CCCAAAGGUUCCCUCAGGCUGAAUGGAAACCAGCCAGAGAGUGUAA...,bpRNA_RFAM_23681


In [102]:
df = pd.merge(bp_rfam, rfam, on='sequence', how='inner')
with open('temp.txt', 'w') as f:
    for n in df['family_name'].unique():
        f.write('{}\n'.format(n))

In [138]:
def clean_family(x):
    if 'mir-' in x or 'MIR' in x or 'lsy-6' in x:
        return 'microRNA'
    if 'rRNA' in x:
        return 'rRNA'
    if 'CbSR' in x:
        return 'sRNA'
    if 'virus' in x.lower():
        return 'virus'
    if 'HACA' in x:
        return 'sRNP'
    if 'RNaseP' in x:
        return 'RNaseP'
    if 'CRISPR' in x:
        return 'crRNA'
    if 'sn' in x:
        return 'snRNA'
    if 'CuYV_BPYV' in x:
        return 'virus'
    if 'SPCSV' in x:
        return 'virus'
    if 'HAV' in x:
        return 'virus'
    if 'STnc' in x:
        return 'sRNA'
    if 'CC' in x:
        return 'sRNA'
    if 'Gl_U' in x:
        return 'snRNA'
    else:
        return x

df['family_name'] = df['family_name'].apply(clean_family)
print(df.groupby('family_name').agg({'family_name': 'count', 'full_family_name':set}).rename(columns={'family_name':'count'}).sort_values('count').head(10))
print(df['family_name'].value_counts().value_counts().head(10))

                count                                   full_family_name
family_name                                                             
RF_site6            2             {Polymerase ribosomal frameshift site}
sn2841              2                       {Small nucleolar RNA sn2841}
TB3Cs2H1            2                   {Trypanosomatid snoRNA TB3Cs2H1}
snoR110             2                      {Small nucleolar RNA snoR110}
SBRMV1_UPD-PKd      2  {Pseudoknot of upstream pseudoknot domain (UPD...
SBRMV1_UPD-PKf      2  {Pseudoknot of upstream pseudoknot domain (UPD...
SBWMV1_UPD-PKb      2  {Pseudoknot of upstream pseudoknot domain (UPD...
Diantho_3_UTR       2               {Dianthovirus RNA2 3'UTR stem loops}
SBWMV1_UPD-PKh      2  {Pseudoknot of upstream pseudoknot domain (UPD...
sR40                2                         {Small nucleolar RNA sR40}
count
5     116
3      90
6      85
4      79
2      78
7      58
8      42
9      35
10     35
20     27
Name: count, dtype